<a href="https://www.kaggle.com/code/rudravohra/notebookdbd8831a05?scriptVersionId=227954131" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rtdetr-35epoch/keras/default/1/epoch35.pt
/kaggle/input/videos/videos/scenevideo_3.mp4
/kaggle/input/videos/videos/scenevideo_11.mp4
/kaggle/input/videos/videos/scenevideo_37.mp4
/kaggle/input/videos/videos/scenevideo_21.mp4
/kaggle/input/videos/videos/scenevideo_18.mp4
/kaggle/input/videos/videos/scenevideo_7.mp4
/kaggle/input/videos/videos/scenevideo_45.mp4
/kaggle/input/videos/videos/scenevideo_5.mp4
/kaggle/input/videos/videos/scenevideo_20.mp4
/kaggle/input/videos/videos/scenevideo_26.mp4
/kaggle/input/videos/videos/scenevideo_22.mp4
/kaggle/input/videos/videos/scenevideo_1.mp4
/kaggle/input/videos/videos/scenevideo_40.mp4
/kaggle/input/videos/videos/scenevideo_23.mp4
/kaggle/input/videos/videos/scenevideo_10.mp4
/kaggle/input/videos/videos/scenevideo_50.mp4
/kaggle/input/videos/videos/scenevideo_43.mp4
/kaggle/input/videos/videos/scenevideo_35.mp4
/kaggle/input/videos/videos/scenevideo_39.mp4
/kaggle/input/videos/videos/scenevideo_17.mp4
/kaggle/input/videos/videos/

In [2]:
!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 17.0 MB/s eta 0:00:00a 0:00:01


In [3]:
import cv2
import os
from ultralytics import YOLO,RTDETR

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = RTDETR("/kaggle/input/rtdetr-best/keras/default/1/best (5).pt").to("cuda")

In [ ]:
videos_path = '/kaggle/input/videos/videos/'
output_dir = "/kaggle/working/testing"  # Output directory for processed videos
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Set up video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLOv8 on the current frame
        results = model(frame, conf=0.15, iou=0.7, max_det=500, agnostic_nms=True,verbose=False)
        annotated_frame = results[0].plot()  # Annotate the frame with detections

        # Write the annotated frame to the output video
        out.write(annotated_frame)

    cap.release()
    out.release()
    print(f"Processed and saved: {output_path}")

In [ ]:
video_files = [f for f in os.listdir(videos_path) if f.endswith('.mp4')]

In [ ]:
len(video_files)

In [ ]:
video_files

In [ ]:
for video in video_files:
    input_path = os.path.join(videos_path, video)
    output_path = os.path.join(output_dir, video.replace('.mp4', '_output.mp4'))
    process_video(input_path, output_path)
print("Processing complete. All videos saved to:", output_dir)    

In [ ]:
process_video("/kaggle/input/videos/videos/scenevideo_36.mp4","/kaggle/working/testing/rtdetr.mp4")

In [ ]:
import shutil

# Path to the folder containing the processed videos
folder_path = '/kaggle/working/processed-videos'
zip_file_path = '/kaggle/working/processed-videos.zip'

# Create a zip file of the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Zipped folder created: {zip_file_path}")


In [ ]:
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    return inter_area / (box1_area + box2_area - inter_area)

In [ ]:
def soft_nms(boxes, scores, sigma=0.8, Nt=0.4, threshold=0.01):
    
    N = len(scores)
    for i in range(N):
        for j in range(i + 1, N):
            iou = compute_iou(boxes[i], boxes[j])
            if iou > Nt:
                scores[j] *= np.exp(-(iou ** 2) / sigma)  # Gaussian decay

    keep = np.where(scores > threshold)[0]
    return keep

In [ ]:
model=YOLO("/kaggle/input/yolov8besr/keras/default/1/best (1).pt")

In [ ]:
def process_video_nms(video_path, output_path):

    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Set up video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Generate unique colors for each class
    num_classes = len(model.names)
    class_colors = {i: tuple(np.random.randint(0, 255, 3).tolist()) for i in range(num_classes)}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLOv8 on the current frame
        results = model(frame,conf=0.18, max_det=500, agnostic_nms=True,verbose=False)

        # Collect boxes, scores, and labels
        boxes, scores, labels = [], [], []
        for r in results:
            for box, score, cls in zip(r.boxes.xyxy.cpu().numpy(), r.boxes.conf.cpu().numpy(), r.boxes.cls.cpu().numpy()):
                boxes.append(box)
                scores.append(score)
                labels.append(cls)

        if len(boxes) > 0:
            # Convert boxes and scores to numpy arrays
            boxes = np.array(boxes)
            scores = np.array(scores)

            # Draw each detection on the frame
            for box, score, label in zip(boxes, scores, labels):
                x1, y1, x2, y2 = map(int, box)
                class_id = int(label)

                # Get class name and color
                class_name = model.names[class_id]
                color = class_colors[class_id]

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Prepare label text and size
                label_text = f"{class_name}: {score:.2f}"
                (text_width, text_height), baseline = cv2.getTextSize(
                    label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2
                )

                # Draw a filled rectangle as the background for the label
                cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)

                # Put the label text on top of the background
                cv2.putText(
                    frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2
                )

        # Write the annotated frame to the output video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Processed and saved: {output_path}")


In [ ]:
video_path = "/kaggle/input/videos/videos/scenevideo_36.mp4"  # Path to input video
output_path = "/kaggle/working/testing/soft-nms.mp4"  # Path to save the output video

In [ ]:
process_video_nms(video_path, output_path)

In [ ]:
import os

# Path to the ZIP file
file_to_remove = "/kaggle/working/yolov8_bifpn.yaml"

# Check if it's a file and remove it
if os.path.isfile(file_to_remove):
    os.remove(file_to_remove)
    print(f"File '{file_to_remove}' has been removed successfully.")
else:
    print(f"File '{file_to_remove}' does not exist or is not a file.")



In [ ]:
import shutil
import os

# Directory to remove
dir_to_remove = "/kaggle/working/testing"

# Check if the directory exists before removing
if os.path.exists(dir_to_remove):
    shutil.rmtree(dir_to_remove)
    print(f"Directory '{dir_to_remove}' has been removed successfully.")
else:
    print(f"Directory '{dir_to_remove}' does not exist.")


In [4]:
!pip install ensemble_boxes

In [5]:
from ensemble_boxes import weighted_boxes_fusion
from ultralytics import RTDETR

In [6]:
# Load YOLOv8 and YOLOv9 models
model_v8 = YOLO("/kaggle/input/yolov8besr/keras/default/1/best (1).pt")  # Replace with your YOLOv8 model path
model_v9 = YOLO("/kaggle/input/yolov9bestt/keras/default/1/yolov9best.pt")  # Replace with your YOLOv9 model path
model_rtdetr = RTDETR("/kaggle/input/rtdetr-best/keras/default/1/best (5).pt")

In [7]:
class_names=['trak', 'cyclist', 'bike', 'tempo', 'car', 'zeep', 'toto', 'e-rickshaw', 'auto-rickshaw', 'bus', 'van', 'cycle-rickshaw', 'person', 'taxi']
class_colors = {
    'trak': (255, 0, 0),            # Red
    'cyclist': (0, 255, 0),         # Green
    'bike': (0, 0, 255),            # Blue
    'tempo': (255, 255, 0),         # Cyan
    'car': (255, 0, 255),           # Magenta
    'zeep': (0, 255, 255),          # Yellow
    'toto': (128, 0, 128),          # Purple
    'e-rickshaw': (255, 165, 0),    # Orange
    'auto-rickshaw': (0, 128, 128), # Teal
    'bus': (128, 128, 0),           # Olive
    'van': (75, 0, 130),            # Indigo
    'cycle-rickshaw': (255, 192, 203), # Pink
    'person': (0, 0, 0),            # Black
    'taxi': (192, 192, 192)         # Silver
}

In [8]:
def filter_large_boxes(boxes, scores, labels, max_area=0.9):
    filtered_boxes = []
    filtered_scores = []
    filtered_labels = []

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        area = (x2 - x1) * (y2 - y1) / (width * height)

        if area <= max_area:
            filtered_boxes.append(box)
            filtered_scores.append(scores[i])
            filtered_labels.append(labels[i])

    return np.array(filtered_boxes), np.array(filtered_scores), np.array(filtered_labels)


**Adaptive-NMS**

In [9]:
import torch,torchvision

In [ ]:
def adaptive_nms(boxes, scores, labels, base_iou=0.01, min_iou=0.05, max_iou=0.95):
    if len(boxes) == 0:
        return np.array([]), np.array([]), np.array([])

    # Convert to torch tensors
    boxes = torch.tensor(boxes, dtype=torch.float32)
    scores = torch.tensor(scores, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.int64)

    # Scale IoU threshold dynamically and convert to tensor
    iou_thresh = torch.tensor(min_iou + (max_iou - min_iou) * scores.numpy(), dtype=torch.float32, device=boxes.device)

    keep_indices = []

    for cls in torch.unique(labels):  # Apply per-class NMS
        class_mask = labels == cls
        class_boxes = boxes[class_mask]
        class_scores = scores[class_mask]
        class_iou_thresh = iou_thresh[class_mask]

        if len(class_boxes) == 1:  # ✅ Handle case with only one box per class
            keep_indices.append(torch.where(class_mask)[0].item())
            continue

        # Sort by confidence
        sorted_indices = torch.argsort(class_scores, descending=True)
        class_boxes = class_boxes[sorted_indices]
        class_scores = class_scores[sorted_indices]
        class_iou_thresh = class_iou_thresh[sorted_indices]

        while len(class_boxes) > 0:
            keep_indices.append(sorted_indices[0].item())  # Keep highest confidence box
            if len(class_boxes) == 1:
                break
            
            # Compute IoU with remaining boxes
            ious = torchvision.ops.box_iou(class_boxes[:1], class_boxes[1:])[0]
            
            # Ensure correct indexing (convert to torch tensor)
            remaining_mask = ious < class_iou_thresh[1:].to(ious.device)

            # ✅ Avoid indexing errors by checking length before slicing
            if len(class_boxes) > 1:
                class_boxes = class_boxes[1:][remaining_mask]
                class_scores = class_scores[1:][remaining_mask]
                class_iou_thresh = class_iou_thresh[1:][remaining_mask]
            else:
                break

    # Apply the kept indices
    return boxes[keep_indices].numpy(), scores[keep_indices].numpy(), labels[keep_indices].numpy()


In [ ]:
def process_frame_adaptive_nms(frame):
    height, width = frame.shape[:2]

    # Convert frame to RGB as required by the models
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get predictions from YOLOv8
    results_v8 = model_v8.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_v8 = results_v8.boxes.xyxy.cpu().numpy()
    scores_v8 = results_v8.boxes.conf.cpu().numpy()
    labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

    # Get predictions from YOLOv9
    results_v9 = model_v9.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_v9 = results_v9.boxes.xyxy.cpu().numpy()
    scores_v9 = results_v9.boxes.conf.cpu().numpy()
    labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)

    # Get predictions from RT-DETR
    results_rtdetr = model_rtdetr.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_rtdetr = results_rtdetr.boxes.xyxy.cpu().numpy()
    scores_rtdetr = results_rtdetr.boxes.conf.cpu().numpy()
    labels_rtdetr = results_rtdetr.boxes.cls.cpu().numpy().astype(int)

    # ✅ Apply filtering on RT-DETR boxes **before** normalizing
    boxes_rtdetr, scores_rtdetr, labels_rtdetr = filter_large_boxes(boxes_rtdetr, scores_rtdetr, labels_rtdetr)

    # ✅ Normalize after filtering
    boxes_v8 = np.clip(boxes_v8 / [width, height, width, height], 0, 1)
    boxes_v9 = np.clip(boxes_v9 / [width, height, width, height], 0, 1)

    # Only normalize RT-DETR boxes if they exist
    if boxes_rtdetr.shape[0] > 0:
        boxes_rtdetr = np.clip(boxes_rtdetr / [width, height, width, height], 0, 1)

    # ✅ Dynamically handle WBF weights
    all_boxes, all_scores, all_labels = [boxes_v8.tolist(), boxes_v9.tolist()], [scores_v8.tolist(), scores_v9.tolist()], [labels_v8.tolist(), labels_v9.tolist()]
    weights = [14, 8]  # Weights for YOLOv8 and YOLOv9

    if boxes_rtdetr.shape[0] > 0:
        all_boxes.append(boxes_rtdetr.tolist())
        all_scores.append(scores_rtdetr.tolist())
        all_labels.append(labels_rtdetr.tolist())
        weights.append(0.5)  # Add RT-DETR weight only if predictions exist

    # Apply Weighted Boxes Fusion
    fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels,
        iou_thr=0.6, skip_box_thr=0.30, weights=weights
    )

    # Apply Adaptive NMS after WBF
    fused_boxes, fused_scores, fused_labels = adaptive_nms(
        fused_boxes, fused_scores, fused_labels    
    )

    # ✅ Handle case where no predictions remain
    if len(fused_boxes) == 0:
        return frame

    conf_thresh = 0.1  # Set confidence threshold again after WBF + NMS
    valid_indices = np.where(fused_scores >= conf_thresh)[0]

    fused_boxes = np.array(fused_boxes)[valid_indices]
    fused_scores = np.array(fused_scores)[valid_indices]
    fused_labels = np.array(fused_labels)[valid_indices]

    # **Denormalize** boxes back to pixel coordinates
    fused_boxes = (fused_boxes * [width, height, width, height]).astype(int)

    # Draw bounding boxes on frame
    for box, score, label in zip(fused_boxes, fused_scores, fused_labels):
        x1, y1, x2, y2 = box
        class_name = class_names[int(label)] if int(label) < len(class_names) else f'Class {int(label)}'
        color = class_colors.get(class_name, (0, 255, 0))  # Default color if missing

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        label_text = f'{class_name}: {score:.2f}'
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    return frame


In [ ]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_36.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing1/test36-adaptive_nms-1.mp4'

In [ ]:
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with WBF
    processed_frame = process_frame_adaptive_nms(frame)

    # Write frame to output video
    out.write(processed_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')

## SOFT-NMS

In [10]:
def compute_iou(box, boxes):
    """Computes IoU between one box and an array of boxes."""
    x1 = np.maximum(box[0], boxes[:, 0])
    y1 = np.maximum(box[1], boxes[:, 1])
    x2 = np.minimum(box[2], boxes[:, 2])
    y2 = np.minimum(box[3], boxes[:, 3])

    inter_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    box_area = (box[2] - box[0]) * (box[3] - box[1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union_area = box_area + boxes_area - inter_area

    return inter_area / (union_area + 1e-6)  # Avoid division by zero

In [11]:
def soft_nms(boxes, scores, labels, sigma=0.62, iou_threshold=0.40, score_threshold=0.12):
    
    if len(boxes) == 0:
        return boxes, scores, labels

    # Convert inputs to NumPy arrays
    boxes = np.array(boxes, dtype=np.float32)
    scores = np.array(scores, dtype=np.float32)
    labels = np.array(labels, dtype=np.int32)

    keep_boxes = []
    keep_scores = []
    keep_labels = []

    while len(scores) > 0:
        # Select the highest confidence box
        max_idx = np.argmax(scores)
        max_box = boxes[max_idx]
        max_score = scores[max_idx]
        max_label = labels[max_idx]

        keep_boxes.append(max_box)
        keep_scores.append(max_score)
        keep_labels.append(max_label)

        # Compute IoU with the remaining boxes
        ious = compute_iou(max_box, boxes)

        # Apply Gaussian penalty to scores based on IoU
        scores *= np.exp(-(ious ** 2) / sigma)

        # Remove low-confidence boxes
        keep_mask = scores >= score_threshold
        boxes = boxes[keep_mask]
        scores = scores[keep_mask]
        labels = labels[keep_mask]

    return np.array(keep_boxes), np.array(keep_scores), np.array(keep_labels)

In [12]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_1.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing1/13march(1).mp4'

In [13]:
def process_frame(frame):
    height, width = frame.shape[:2]

    # Convert frame to RGB as required by the models
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get predictions from YOLOv8
    results_v8 = model_v8.predict(img_rgb, verbose=False, agnostic_nms=False)[0]
    boxes_v8 = results_v8.boxes.xyxy.cpu().numpy()
    scores_v8 = results_v8.boxes.conf.cpu().numpy()
    labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

    # Get predictions from YOLOv9
    results_v9 = model_v9.predict(img_rgb, verbose=False, agnostic_nms=False)[0]
    boxes_v9 = results_v9.boxes.xyxy.cpu().numpy()
    scores_v9 = results_v9.boxes.conf.cpu().numpy()
    labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)

    # Get predictions from RT-DETR
    results_rtdetr = model_rtdetr.predict(img_rgb, verbose=False, agnostic_nms=False,conf=0.8,iou=0.2)[0]
    boxes_rtdetr = results_rtdetr.boxes.xyxy.cpu().numpy()
    scores_rtdetr = results_rtdetr.boxes.conf.cpu().numpy()
    labels_rtdetr = results_rtdetr.boxes.cls.cpu().numpy().astype(int)

    # ✅ Apply filtering on RT-DETR boxes **before** normalizing
    boxes_rtdetr, scores_rtdetr, labels_rtdetr = filter_large_boxes(boxes_rtdetr, scores_rtdetr, labels_rtdetr)

    # ✅ Normalize after filtering
    boxes_v8 = np.clip(boxes_v8 / [width, height, width, height], 0, 1)
    boxes_v9 = np.clip(boxes_v9 / [width, height, width, height], 0, 1)

    if boxes_rtdetr.shape[0] > 0:
        boxes_rtdetr = np.clip(boxes_rtdetr / [width, height, width, height], 0, 1)

    # ✅ Dynamically handle WBF weights
    all_boxes, all_scores, all_labels = [boxes_v8.tolist(), boxes_v9.tolist()], [scores_v8.tolist(), scores_v9.tolist()], [labels_v8.tolist(), labels_v9.tolist()]
    weights = [40, 5]

    if boxes_rtdetr.shape[0] > 0:
        all_boxes.append(boxes_rtdetr.tolist())
        all_scores.append(scores_rtdetr.tolist())
        all_labels.append(labels_rtdetr.tolist())
        weights.append(0.00000001)

    # Apply Weighted Boxes Fusion
    fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels,
        iou_thr=0.63, skip_box_thr=0.26, weights=weights
    )

    conf_thresh = 0.17
    valid_indices = np.where(fused_scores >= conf_thresh)[0]

    fused_boxes = np.array(fused_boxes)[valid_indices]
    fused_scores = np.array(fused_scores)[valid_indices]
    fused_labels = np.array(fused_labels)[valid_indices]

    # **Denormalize** boxes back to pixel coordinates
    fused_boxes = (fused_boxes * [width, height, width, height]).astype(int)

    # ✅ Apply Soft-NMS **Here**
    fused_boxes, fused_scores, fused_labels = soft_nms(
        fused_boxes, fused_scores, fused_labels,
       score_threshold=conf_thresh
    )
# Draw bounding boxes on frame
    for box, score, label in zip(fused_boxes, fused_scores, fused_labels):
        x1, y1, x2, y2 = map(int, box)  # Ensure coordinates are integers
        
        class_name = class_names[int(label)] if int(label) < len(class_names) else f'Class {int(label)}'
        
        color = class_colors.get(class_name, (0, 255, 0))  # Default color if missing
        if not isinstance(color, tuple) or len(color) != 3:
            color = (0, 255, 0)  # Fallback to green
        color = tuple(map(int, color))  # Ensure valid color format
        
        #print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}, color: {color}")  # Debugging
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)  # ✅ No error
        label_text = f'{class_name}: {score:.2f}'
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
    return frame    
        

In [ ]:
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with WBF
    processed_frame = process_frame(frame)

    # Write frame to output video
    out.write(processed_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')

### Tracking

In [ ]:
!pip install deep_sort_realtime
!pip install supervision

In [ ]:
import supervision as sv

In [ ]:
from ensemble_boxes import weighted_boxes_fusion

In [ ]:
byte_tracker = sv.ByteTrack(
    track_activation_threshold=0.01,  # ✅ Lower threshold to keep more objects
    lost_track_buffer=30,
    minimum_matching_threshold=0.1,  # ✅ Allow looser matches
    frame_rate=30,
    minimum_consecutive_frames=1  # ✅ No need to see object multiple times
)

In [ ]:
box_annotator = sv.BoxAnnotator(thickness=3)
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1.5, text_color=sv.Color.BLACK)

In [ ]:
def process_frame_track(frame):
    height, width = frame.shape[:2]
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLOv8 predictions
    results_v8 = model_v8.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_v8 = results_v8.boxes.xyxy.cpu().numpy()
    scores_v8 = results_v8.boxes.conf.cpu().numpy()
    labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

    # YOLOv9 predictions
    results_v9 = model_v9.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_v9 = results_v9.boxes.xyxy.cpu().numpy()
    scores_v9 = results_v9.boxes.conf.cpu().numpy()
    labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)

    # Normalize boxes
    boxes_v8 = np.clip(boxes_v8 / [width, height, width, height], 0, 1)
    boxes_v9 = np.clip(boxes_v9 / [width, height, width, height], 0, 1)

    # Apply Weighted Boxes Fusion
    all_boxes = [boxes_v8.tolist(), boxes_v9.tolist()]
    all_scores = [scores_v8.tolist(), scores_v9.tolist()]
    all_labels = [labels_v8.tolist(), labels_v9.tolist()]
    weights = [10, 8]

    fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels, iou_thr=0.63, skip_box_thr=0.26, weights=weights
    )

    conf_thresh = 0.17
    valid_indices = np.where(fused_scores >= conf_thresh)[0]
    fused_boxes = np.array(fused_boxes)[valid_indices]
    fused_scores = np.array(fused_scores)[valid_indices]
    fused_labels = np.array(fused_labels)[valid_indices]

    # Denormalize boxes
    fused_boxes = (fused_boxes * [width, height, width, height]).astype(int)

    # Convert detections to Supervision format
    detections = sv.Detections(
        xyxy=fused_boxes,
        confidence=fused_scores,
        class_id=fused_labels.astype(int)  # ✅ Ensure class_id is integer
    )

    # 🔍 Debugging: Print before tracking
    print(f"Before Tracking: {len(detections)} objects detected by YOLO (after WBF)")

    # Apply ByteTrack
    detections = byte_tracker.update_with_detections(detections)

    # 🔍 Debugging: Print after tracking
    print(f"After Tracking: {len(detections)} objects remaining after ByteTrack")

    # Format labels
    labels = [
        f"#{tracker_id} {model_v8.model.names[class_id]} {confidence:.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]

    # Draw detections
    annotated_frame = frame.copy()
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    return annotated_frame


In [ ]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_36.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing/test36-(1)-bytetracker.mp4'

In [ ]:
 #Open video
cap = cv2.VideoCapture(input_video_path)
frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame_track(frame)
    out.write(processed_frame)

# Release resources
cap.release()
out.release()

print(f"✅ Processed video saved at: {output_video_path}")

In [ ]:
!pip install deep-sort-realtime

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import torch

In [ ]:
deep_sort_tracker = DeepSort(
    max_age=5,           # Remove lost tracks faster
    embedder="mobilenet",
    n_init=4,             # Require more detections before confirming a track
    max_iou_distance=0.72  # Keep stable, or slightly increase if needed
)

In [ ]:
def process_frame_track_deepsort(frame):
    height, width = frame.shape[:2]
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLOv8 predictions
    results_v8 = model_v8.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_v8 = results_v8.boxes.xyxy.cpu().numpy()
    scores_v8 = results_v8.boxes.conf.cpu().numpy()
    labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

    # YOLOv9 predictions
    results_v9 = model_v9.predict(img_rgb, verbose=False, agnostic_nms=True)[0]
    boxes_v9 = results_v9.boxes.xyxy.cpu().numpy()
    scores_v9 = results_v9.boxes.conf.cpu().numpy()
    labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)

    # Normalize boxes
    boxes_v8 = np.clip(boxes_v8 / [width, height, width, height], 0, 1)
    boxes_v9 = np.clip(boxes_v9 / [width, height, width, height], 0, 1)

    # Apply Weighted Boxes Fusion
    all_boxes = [boxes_v8.tolist(), boxes_v9.tolist()]
    all_scores = [scores_v8.tolist(), scores_v9.tolist()]
    all_labels = [labels_v8.tolist(), labels_v9.tolist()]
    weights = [10, 8]

    fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels, iou_thr=0.63, skip_box_thr=0.26, weights=weights
    )

    conf_thresh = 0.17
    valid_indices = np.where(fused_scores >= conf_thresh)[0]
    fused_boxes = np.array(fused_boxes)[valid_indices]
    fused_scores = np.array(fused_scores)[valid_indices]
    fused_labels = np.array(fused_labels)[valid_indices]

    # Denormalize boxes
    fused_boxes = (fused_boxes * [width, height, width, height]).astype(int)

    # Convert detections to DeepSORT format
    deepsort_detections = []
    for box, score, label in zip(fused_boxes, fused_scores, fused_labels):
        x1, y1, x2, y2 = box
        deepsort_detections.append(([x1, y1, x2 - x1, y2 - y1], score, label))

    # Apply DeepSORT tracking
    tracked_objects = deep_sort_tracker.update_tracks(deepsort_detections, frame=frame)

    # Prepare tracking results
    detections = []
    tracked_labels = []  # ✅ Store class_id for Supervision
    labels = []
    for track in tracked_objects:
        if not track.is_confirmed():
            continue
        x1, y1, x2, y2 = track.to_ltrb()
        detections.append([x1, y1, x2, y2])
        tracked_labels.append(track.det_class)  # ✅ Store class_id
        labels.append(f"ID {track.track_id} {model_v8.model.names[track.det_class]}")

    annotated_frame = frame.copy()

    if len(detections) > 0:
        detections = sv.Detections(
            xyxy=np.array(detections),
            class_id=np.array(tracked_labels, dtype=int)  # ✅ Include class_id
        )
    
        annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
        annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)


    return annotated_frame


In [ ]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_36.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing/test36-(6)-deepsort.mp4'

In [ ]:
# Open video
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    processed_frame = process_frame_track_deepsort(frame)
    out.write(processed_frame)

cap.release()
out.release()

print(f"Processed video saved to: {output_video_path}")


# 3/13/25

In [14]:
def process_frame(frame):
    height, width = frame.shape[:2]
    frame_area = width * height  # Total frame area

    # === Step 1: Get YOLOv8 Predictions ===
    results_v8 = model_v8.predict(frame, verbose=False, agnostic_nms=False, conf=0.25)[0]
    boxes_v8 = results_v8.boxes.xyxy.cpu().numpy() / [width, height, width, height]  # Normalize
    scores_v8 = results_v8.boxes.conf.cpu().numpy()
    labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

    # === Step 2: Get YOLOv9 Predictions ===
    results_v9 = model_v9.predict(frame, verbose=False, agnostic_nms=False, conf=0.25)[0]
    boxes_v9 = results_v9.boxes.xyxy.cpu().numpy() / [width, height, width, height]  # Normalize
    scores_v9 = results_v9.boxes.conf.cpu().numpy()
    labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)

    # === Step 3: Apply Weighted Box Fusion (WBF) ===
    all_boxes = [boxes_v8, boxes_v9]
    all_scores = [scores_v8, scores_v9]
    all_labels = [labels_v8, labels_v9]
    weights = [1.2, 1.8]  # Equal weight for both models

    boxes_wbf, scores_wbf, labels_wbf = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels, weights=weights, iou_thr=0.5, skip_box_thr=0.1
    )

    # Convert back to original scale
    boxes_wbf *= [width, height, width, height]
    boxes_wbf = boxes_wbf.astype(int)
    labels_wbf = labels_wbf.astype(int)  # Ensure integer labels

    # === Step 4: Apply Soft-NMS ===
    boxes_final, scores_final, labels_final = soft_nms(boxes_wbf, scores_wbf, labels_wbf)

    # === Step 5: Filter by Confidence Threshold (>= 0.16) and Remove Large Screen-Filling Boxes ===
    valid_indices = []
    for i, (x1, y1, x2, y2) in enumerate(boxes_final):
        box_area = (x2 - x1) * (y2 - y1)
        if scores_final[i] >= 0.16 and box_area < 0.85 * frame_area:  # Remove huge boxes (> 85% screen area)
            valid_indices.append(i)

    boxes_final = boxes_final[valid_indices]
    scores_final = scores_final[valid_indices]
    labels_final = labels_final[valid_indices]

    # === Step 6: Draw Final Detections ===
    for box, score, label in zip(boxes_final, scores_final, labels_final):
        x1, y1, x2, y2 = map(int, box)  # Ensure all coordinates are integers
        
        # Clamp coordinates to frame boundaries
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
        
        if x2 <= x1 or y2 <= y1:
            continue  # Skip invalid boxes

        # Get class name
        class_name = class_names[label] if 0 <= label < len(class_names) else f"Class_{label}"

        # Get color
        color = class_colors.get(class_name, (0, 255, 0))  # Default green
        color = tuple(map(int, color))  # Ensure integer values

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Label text
        label_text = f"{class_name}: {score:.2f}"
        
        # Text size
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)

        # Draw text background
        cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)

        # Draw text
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, lineType=cv2.LINE_AA)

    return frame


In [14]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_36.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing1/13march(36-modification-1).mp4'

In [ ]:
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with WBF
    processed_frame = process_frame(frame)

    # Write frame to output video
    out.write(processed_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')

 # Test Time Augmentation # 

In [ ]:
import albumentations as A

In [ ]:
# Define TTA transformations
tta_transforms = [
    lambda img: img,  # Original
    lambda img: cv2.flip(img, 1),  # Horizontal Flip
    lambda img: A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10, p=1)(image=img)['image'],  # Small Rotate & Scale
]

def reverse_tta_boxes(boxes, width, height, tta_index):
    """Reverse the augmentation applied to the bounding boxes."""
    if tta_index == 1:  # Horizontal Flip
        boxes[:, [0, 2]] = width - boxes[:, [2, 0]]  # Flip x-coordinates
    return boxes

In [ ]:
def process_frame(frame):
    height, width = frame.shape[:2]
    frame_area = width * height  

    all_boxes_v8, all_scores_v8, all_labels_v8 = [], [], []
    all_boxes_v9, all_scores_v9, all_labels_v9 = [], [], []

    # === Step 1: Run TTA for YOLOv8 & YOLOv9 ===
    for tta_index, transform in enumerate(tta_transforms):
        aug_frame = transform(frame.copy())

        # YOLOv8
        results_v8 = model_v8.predict(aug_frame, verbose=False, agnostic_nms=False, conf=0.25)[0]
        boxes_v8 = results_v8.boxes.xyxy.cpu().numpy() / [width, height, width, height]  
        scores_v8 = results_v8.boxes.conf.cpu().numpy()
        labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

        # YOLOv9
        results_v9 = model_v9.predict(aug_frame, verbose=False, agnostic_nms=False, conf=0.25)[0]
        boxes_v9 = results_v9.boxes.xyxy.cpu().numpy() / [width, height, width, height]  
        scores_v9 = results_v9.boxes.conf.cpu().numpy()
        labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)

        # Reverse TTA for boxes
        boxes_v8 = reverse_tta_boxes(boxes_v8, width, height, tta_index)
        boxes_v9 = reverse_tta_boxes(boxes_v9, width, height, tta_index)

        # Store results
        all_boxes_v8.append(boxes_v8), all_scores_v8.append(scores_v8), all_labels_v8.append(labels_v8)
        all_boxes_v9.append(boxes_v9), all_scores_v9.append(scores_v9), all_labels_v9.append(labels_v9)

    # === Step 2: Apply Weighted Box Fusion (WBF) ===
    weights = [1.2, 1.2, 1.2, 1.8, 1.8, 1.8]  # 2 models × 3 augmentations

    boxes_wbf, scores_wbf, labels_wbf = weighted_boxes_fusion(
    all_boxes_v8 + all_boxes_v9, 
    all_scores_v8 + all_scores_v9, 
    all_labels_v8 + all_labels_v9, 
    weights=weights, iou_thr=0.5, skip_box_thr=0.1
)

    # Convert to original scale
    boxes_wbf *= [width, height, width, height]
    boxes_wbf = boxes_wbf.astype(int)
    labels_wbf = labels_wbf.astype(int)

    # === Step 3: Apply Soft-NMS ===
    boxes_final, scores_final, labels_final = soft_nms(boxes_wbf, scores_wbf, labels_wbf)

    # === Step 4: Filter by Confidence & Remove Large Boxes ===
    valid_indices = []
    for i, (x1, y1, x2, y2) in enumerate(boxes_final):
        box_area = (x2 - x1) * (y2 - y1)
        if scores_final[i] >= 0.16 and box_area < 0.85 * frame_area:
            valid_indices.append(i)

    boxes_final = boxes_final[valid_indices]
    scores_final = scores_final[valid_indices]
    labels_final = labels_final[valid_indices]

    # === Step 5: Draw Final Detections ===
    for box, score, label in zip(boxes_final, scores_final, labels_final):
        x1, y1, x2, y2 = map(int, box)  
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)

        if x2 <= x1 or y2 <= y1:
            continue  

        class_name = class_names[label] if 0 <= label < len(class_names) else f"Class_{label}"
        color = class_colors.get(class_name, (0, 255, 0))  
        color = tuple(map(int, color))  

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        label_text = f"{class_name}: {score:.2f}"
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, lineType=cv2.LINE_AA)

    return frame

In [ ]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_36.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing1/13march(36-tta-2).mp4'

In [ ]:
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with WBF
    processed_frame = process_frame(frame)

    # Write frame to output video
    out.write(processed_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')

# SAHI

In [15]:
import cv2

In [16]:
!pip install sahi ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=bde872a5d0e854c5906a177590f7c8ed87ea4158dd74a8fef69c24fc473f1a42
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [17]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import torch

In [ ]:
def process_frame(frame):
    height, width = frame.shape[:2]
    frame_area = width * height
    alpha = 0.6  # From YOSCA paper [Source 2]

    # ===== 0. Load Original Models for Baseline =====
    model_v8 = YOLO("/kaggle/input/yolov8besr/keras/default/1/best (1).pt")  # Original YOLOv8
    model_v9 = YOLO("/kaggle/input/yolov9bestt/keras/default/1/yolov9best.pt")  # Original YOLOv9

    # ===== 1. SAHI-Enhanced YOLOv8 Predictions =====
    sahiv8_model = AutoDetectionModel.from_pretrained(
        model_type="yolov8",
        model_path="/kaggle/input/yolov8besr/keras/default/1/best (1).pt",
        confidence_threshold=0.25,
        device="cuda",
        image_size=640,
    )
    results_v8 = get_sliced_prediction(
        frame, sahiv8_model,
        slice_height=896, slice_width=896,
        overlap_height_ratio=0.2, overlap_width_ratio=0.2,
        verbose=0, postprocess_match_threshold=0.5
    )

    # Get baseline YOLOv8 predictions
    baseline_v8 = model_v8.predict(frame, verbose=False, conf=0.25, imgsz=640)[0]
    baseline_scores_v8 = baseline_v8.boxes.conf.cpu().numpy()
    mean_baseline_v8 = baseline_scores_v8.mean() if len(baseline_scores_v8) > 0 else 0.0

    # Handle SAHI predictions with YOSCA adjustment
    if len(results_v8.object_prediction_list) > 0:
        boxes_v8 = np.array([obj.bbox.to_xyxy() for obj in results_v8.object_prediction_list]) / [width, height, width, height]
        boxes_v8 = boxes_v8.reshape(-1, 4)
        scores_v8 = np.array([obj.score.value for obj in results_v8.object_prediction_list])
        scores_v8 = alpha * scores_v8 + (1 - alpha) * mean_baseline_v8  # YOSCA blending
        labels_v8 = np.array([obj.category.id for obj in results_v8.object_prediction_list])
    else:
        boxes_v8 = np.empty((0, 4))
        scores_v8 = np.array([])
        labels_v8 = np.array([])

    # ===== 2. SAHI-Enhanced YOLOv9 Predictions ===== 
    sahiv9_model = AutoDetectionModel.from_pretrained(
        model_type="ultralytics",
        model_path="/kaggle/input/yolov9bestt/keras/default/1/yolov9best.pt",
        confidence_threshold=0.25,
        device="cuda",
        image_size=640
    )
    results_v9 = get_sliced_prediction(
        frame, sahiv9_model,
        slice_height=896, slice_width=896,
        overlap_height_ratio=0.2, overlap_width_ratio=0.2,
        verbose=0, postprocess_match_threshold=0.5
    )

    # Get baseline YOLOv9 predictions
    baseline_v9 = model_v9.predict(frame, verbose=False, conf=0.25, imgsz=640)[0]
    baseline_scores_v9 = baseline_v9.boxes.conf.cpu().numpy()
    mean_baseline_v9 = baseline_scores_v9.mean() if len(baseline_scores_v9) > 0 else 0.0

    if len(results_v9.object_prediction_list) > 0:
        boxes_v9 = np.array([obj.bbox.to_xyxy() for obj in results_v9.object_prediction_list]) / [width, height, width, height]
        boxes_v9 = boxes_v9.reshape(-1, 4)
        scores_v9 = np.array([obj.score.value for obj in results_v9.object_prediction_list])
        scores_v9 = alpha * scores_v9 + (1 - alpha) * mean_baseline_v9  # YOSCA blending
        labels_v9 = np.array([obj.category.id for obj in results_v9.object_prediction_list])
    else:
        boxes_v9 = np.empty((0, 4))
        scores_v9 = np.array([])
        labels_v9 = np.array([])
                             
    # === Step 3: Apply Weighted Box Fusion (WBF) ===
    all_boxes = [boxes_v8, boxes_v9]
    all_scores = [scores_v8, scores_v9]
    all_labels = [labels_v8, labels_v9]
    weights = [1.2, 1.8]  

    boxes_wbf, scores_wbf, labels_wbf = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels, weights=weights, iou_thr=0.5, skip_box_thr=0.1
    )

    # Convert back to original scale
    boxes_wbf *= [width, height, width, height]
    boxes_wbf = boxes_wbf.astype(int)
    labels_wbf = labels_wbf.astype(int)  # Ensure integer labels

    # === Step 4: Apply Soft-NMS ===
    boxes_final, scores_final, labels_final = soft_nms(boxes_wbf, scores_wbf, labels_wbf)

    # === Step 5: Filter by Confidence Threshold (>= 0.16) and Remove Large Screen-Filling Boxes ===
    valid_indices = []
    for i, (x1, y1, x2, y2) in enumerate(boxes_final):
        box_area = (x2 - x1) * (y2 - y1)
        if scores_final[i] >= 0.16 and box_area < 0.85 * frame_area:  # Remove huge boxes (> 85% screen area)
            valid_indices.append(i)

    boxes_final = boxes_final[valid_indices]
    scores_final = scores_final[valid_indices]
    labels_final = labels_final[valid_indices]

    # === Step 6: Draw Final Detections ===
    for box, score, label in zip(boxes_final, scores_final, labels_final):
        x1, y1, x2, y2 = map(int, box)  # Ensure all coordinates are integers
        
        # Clamp coordinates to frame boundaries
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
        
        if x2 <= x1 or y2 <= y1:
            continue  # Skip invalid boxes

        # Get class name
        class_name = class_names[label] if 0 <= label < len(class_names) else f"Class_{label}"

        # Get color
        color = class_colors.get(class_name, (0, 255, 0))  # Default green
        color = tuple(map(int, color))  # Ensure integer values

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Label text
        label_text = f"{class_name}: {score:.2f}"
        
        # Text size
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)

        # Draw text background
        cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)

        # Draw text
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, lineType=cv2.LINE_AA)

    return frame

In [ ]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_36.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing1/15march(36-modification-2).mp4'

In [ ]:
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with WBF
    processed_frame = process_frame(frame)

    # Write frame to output video
    out.write(processed_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    
    # Enclose the path in double quotes to handle special characters
    command = f'zip -r "{zip_name}" "{path}"'

    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    
    display(FileLink(f"{download_file_name}.zip"))

# Call the function correctly
download_file('/kaggle/working/testing1/15march(36-modification-2).mp4', 'out')


# Shadows

In [27]:
!pip install CuPy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for CuPy
  Running setup.py clean for CuPy
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for CuPy
Failed to build CuPy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (CuPy)


In [29]:
def adaptive_gamma_correction(image, gamma_min=0.5, gamma_max=2.5):
    """Automatically adjusts gamma based on image brightness."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean_intensity = np.mean(gray) / 255.0  # Normalize to [0, 1]

    # Compute adaptive gamma (inverse relationship with brightness)
    gamma = gamma_max - (gamma_max - gamma_min) * mean_intensity

    # Apply gamma correction
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    corrected = cv2.LUT(image, table)

    return corrected

In [51]:
def process_frame(frame):
    height, width = frame.shape[:2]
    frame_area = width * height  

    # === Step 1: Apply Adaptive Gamma Correction (AGC) ===
    frame = adaptive_gamma_correction(frame)  # Already implemented by you

    # === Step 2: Get YOLOv8 Predictions ===
    results_v8 = model_v8.predict(frame, verbose=False, agnostic_nms=False, conf=0.25)[0]
    boxes_v8 = results_v8.boxes.xyxy.cpu().numpy()
    scores_v8 = results_v8.boxes.conf.cpu().numpy()
    labels_v8 = results_v8.boxes.cls.cpu().numpy().astype(int)

    # === Step 3: Get YOLOv9 Predictions ===
    results_v9 = model_v9.predict(frame, verbose=False, agnostic_nms=False, conf=0.25)[0]
    boxes_v9 = results_v9.boxes.xyxy.cpu().numpy()
    scores_v9 = results_v9.boxes.conf.cpu().numpy()
    labels_v9 = results_v9.boxes.cls.cpu().numpy().astype(int)
    
    # === Step 4: Apply Weighted Box Fusion (WBF) ===
    # Normalize YOLOv8 boxes
    boxes_v8[:, [0, 2]] /= width   # Normalize X coordinates
    boxes_v8[:, [1, 3]] /= height  # Normalize Y coordinates
    
    # Normalize YOLOv9 boxes
    boxes_v9[:, [0, 2]] /= width
    boxes_v9[:, [1, 3]] /= height
    
    all_boxes = [boxes_v8, boxes_v9]
    all_scores = [scores_v8, scores_v9]
    all_labels = [labels_v8, labels_v9]
    weights = [1.2, 1.8]

    boxes_wbf, scores_wbf, labels_wbf = weighted_boxes_fusion(
        all_boxes, all_scores, all_labels, weights=weights, iou_thr=0.5, skip_box_thr=0.1
    )

    # Convert back to original scale
    boxes_wbf = boxes_wbf.astype(int)
    labels_wbf = labels_wbf.astype(int)  

    # === Step 5: Apply Soft-NMS ===
    boxes_final, scores_final, labels_final = soft_nms(boxes_wbf, scores_wbf, labels_wbf)

    # === Step 6: Filter by Confidence Threshold (>= 0.16) and Remove Large Boxes ===
    valid_indices = []
    for i, (x1, y1, x2, y2) in enumerate(boxes_final):
        box_area = (x2 - x1) * (y2 - y1)
        if scores_final[i] >= 0.16 and box_area < 0.85 * frame_area:  
            valid_indices.append(i)

    boxes_final = boxes_final[valid_indices]
    scores_final = scores_final[valid_indices]
    labels_final = labels_final[valid_indices]

    # === Step 7: Draw Final Detections ===
    for box, score, label in zip(boxes_final, scores_final, labels_final):
        x1, y1, x2, y2 = map(int, box)
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
        
        if x2 <= x1 or y2 <= y1:
            continue  

        class_name = class_names[label] if 0 <= label < len(class_names) else f"Class_{label}"
        color = class_colors.get(class_name, (0, 255, 0))  
        color = tuple(map(int, color))

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        label_text = f"{class_name}: {score:.2f}"
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)

        cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
        cv2.putText(frame, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, lineType=cv2.LINE_AA)

    return frame


In [52]:
# Video processing
input_video_path = '/kaggle/input/videos/videos/scenevideo_2.mp4'  # Replace with your input video path
output_video_path = '/kaggle/working/testing1/15march(2(1)-modification-shadow).mp4'

In [50]:
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with WBF
    processed_frame = process_frame(frame)

    # Write frame to output video
    out.write(processed_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.